## Chatbot Multi-Agente para Ensino Adaptativo de Tópicos em Deep Learning

### Criação dos Agentes Professor, Dean e Aluno

Objetivos:
  - Estabelecer um plano de ensino introdutório e experimental pelo Dean;
  - Estabelecer interação entre o Professor e o Dean para estabelecer interações com o Aluno;
  - Estabelecer papel do Aluno.

Nome: **Hiuri Santana de Noronha** - RA: **229961**

In [ ]:
#!pip install -U -q openai
!pip install -U -q litellm
!pip install -U -q langchain langchain-community langchain-core
!pip install -U -q langchain-huggingface langchain-openai
!pip install -U -q crewai crewai[tools]
!pip install -U -q tqdm
!pip install -U -q gradio
#!pip install graphviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 10.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently ta

In [ ]:
import os
import requests
import json
from google.colab import userdata
from crewai import Agent, Task, Crew, Process, LLM
from crewai.memory.memory import Memory
from crewai.memory.storage.interface import Storage

#from langchain_openai import ChatOpenAI
#from langchain_community.llms import Ollama

from IPython.display import display, Markdown
from textwrap import dedent
import openai

# Warning control
import warnings
warnings.filterwarnings('ignore')

try:
    os.environ["OLLAMA_API_BASE"] = userdata.get('OLLAMA_API_BASE')
    print("✅ OLLAMA API configurada!")

    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ OpenAI API Key configurada!")

    OLLAMA_BASE_URL = os.environ["OLLAMA_API_BASE"]# + "/v1"
except Exception as e:
    print("⚠️ Erro nas configurações base.", e)

# Ollama model (e.g., gpt-oss:120b, gpt-oss:20b)
ollama_model = "ollama/gpt-oss:120b"

# GPT model (e.g., gpt-5-nano, gpt-5-mini, gpt-5)
gpt_model = "gpt-5-nano"

⚠️ Erro nas configurações base. Requesting secret OLLAMA_API_BASE timed out. Secrets can only be fetched when running from the Colab UI.


### Creating Experimental Agents

In [ ]:
llm = LLM(
    model=ollama_model,
    base_url=OLLAMA_BASE_URL,
    temperature=0.7
)

# Ollama self-hosted sol.
#llm = Ollama( # Não funciona com o CrewAI
#    model=ollama_model,
#    base_url=os.environ["OLLAMA_API_BASE"],
#    temperature=0.7
#)

#llm = ChatOpenAI(
#    model=ollama_model,
#    temperature=0.7,
#    base_url=OLLAMA_BASE_URL
#)

# OpenAI API
#llm = ChatOpenAI(
#    model=gpt_model, # modelo no Ollama remoto
#    temperature=0.7
#)

NameError: name 'OLLAMA_BASE_URL' is not defined

In [ ]:
#@title Tópicos da Simulação
topico_geral = "Deep Learning" #@param ["Deep Learning"]
topico_especifico = "Regressão Linear" #@param ["Regressão Linear", "Redes Neurais Convolucionais (CNNs)", "Redes Neurais Recorrentes (RNNs)"]
print(f"Tema: {topico_geral}")
print(f"Tópico Específico: {topico_especifico}")

Tema: Deep Learning
Tópico Específico: Regressão Linear


In [ ]:
# Storage da memória
class CommonStorage(Storage):
    def __init__(self):
        self.data = []

    def save(self, value, metadata=None):
        self.data.append({"value": value, "metadata": metadata})

    def load(self):
        return self.data

    def clear(self):
        self.data = []

# Instancia do armazenamento da memória
storage = CommonStorage()
memory = Memory(storage=storage)

# Exemplo de uso
memory.save("O aluno estudou Redes Neurais hoje.")
memory.save("Ele quer revisar o tema amanhã.")

print("🧠 Memória atual:")
for item in memory.storage.load():  # ou memory.storage.load()
    print(f"- {item['value']} (meta: {item['metadata']})")

🧠 Memória atual:
- O aluno estudou Redes Neurais hoje. (meta: {})
- Ele quer revisar o tema amanhã. (meta: {})


In [ ]:
# O DEAN
dean_storage = CommonStorage()
dean_memory = Memory(storage=dean_storage)

dean = Agent(
    role='Diretor Acadêmico e Orquestrador (Dean)',
    goal=(
        "Criar plano de ensino sobre o assunto: {topic}. "
        "Analisar o histórico de interações entre agente professor e agente aluno, "
        "com o objetivo de analisar o avanço do aprendizado e o cumprimento do plano de ensino, "
        "e decidir o PRÓXIMO PASSO de interação. Você deve orquestrar a conversa entre o "
        "Professor e o Aluno. Você também decide quando um tópico termina "
        "ou quando a sessão inteira deve ser encerrada. "
        "Crie e grave em sua memória a lista de tópicos "
        "a serem cumpridos do plano de ensino, inicialize com ☐ (referente a incomplete). Você possui três status: "
        "✅ (completed), "
        "☐ (incomplete), "
        "❌ (learning_problem). "
        "Quando lhe for solicitado para responder no formato JSON, responda EXCLUSIVAMENTE em formato JSON válido, conforme instrução solicitada (como prioridade máxima para a resposta)."
    ),
    backstory=(
        "Você é o orquestrador mestre. Você avalia o histórico e decide quem "
        "deve falar em seguida (Professor ou Aluno) e qual é a sua instrução. "
        "Se o aluno aprendeu o tópico, você pode declarar o tópico como 'concluído' e "
        "encerrar a sessão."
    ),
    memory=dean_memory,
    verbose=True,
    llm=llm,
)

In [ ]:
# O PROFESSOR
professor = Agent(
    role='Professor Especialista',
    goal=(
        "Ensinar o tópico de {topic} ao agente aluno, seguindo as instruções do Dean via método Socrático. "
        "Você é submisso ao agente Dean que irá instruir quais os seus passos em sala de aula para o ensino. "
        "Importante saber que na primeira interação com o agente aluno deve elaborar o raciocínio acerca do tema {topic}, "
        "em seguida deve decidir se faz perguntas de multipla escolha ou dissertativa para o aluno, deve avaliar o desempenho, "
        "se o desempenho for insatisfatório, elabore novamente um plano de aula para interagir com o aluno até que o desempenho "
        "seja satisfatório, isto é, o aluno acerte de forma adequada a questão. Proceda neste loop de aula e teste até que "
        "o aproveitamento do aluno seja obtido. Após isso reporte para o Dean o status atual do aprendizado (satisfatório ou não satisfatório)."
    ),
    backstory=(
        "Você é um professor especialista Socrático. Você NUNCA dá a resposta direta. "
        "Você responde perguntas com outras perguntas. Você segue a instrução exata "
        "dada pelo Dean em cada turno. Você elabora aulas e as executa acerca de um tópico."
    ),
    verbose=True,
    llm=llm
)

In [ ]:
# O ALUNO
aluno = Agent(
    role='Aluno Agentico',
    goal="Aprender sobre {topic} e interagir com o Professor.",
    backstory=(
        "Você é um estudante curioso e iniciante, não sabe NADA sobre o {topic}. "
        "Você reage às explicações do professor. Se você não entende, você pergunta. "
        "Você tenta responder às perguntas, mesmo que erre. Porém está em um processo de aprendizado e "
        "a cada vez que interage com o professor tende a convergir com os raciocínios."
    ),
    memory=True, # memória ativada para o agente aluno para estimular o aprendizado
    verbose=True,
    llm=llm
)

In [ ]:
# Tarefa de Setup: Criar o plano
task_setup_plano = Task(
    description=(
        f"Criar o plano de ensino para o curso de '{topico_geral}' sobre o tópico específico de '{topico_especifico}'. "
        "O plano deve conter os tópicos necessários para o aprendizado, seguindo um raciocínio lógico para determinar a progressão, "
        "estabelecendo sempre do mais básico ao avançado sobre o tópico específico. Para a avaliação é importante criar "
        "em memória a lista de tópicos a serem cumpridos do plano de ensino, inicialize com ☐ (referente a incomplete). "
        "Você possui três status: ✅ (complete), ☐ (incomplete), ❌ (learning_problem). "
    ),
    expected_output=(
        "Pode discorrer sobre o plano de ensino."
        "Deve mostrar um planejamento com uma lista formatada sobre os tópicos de ensino, e.g., '- Tópico 1\n- Tópico 2...'. "
        "Formato de saída: "
        "[PLANO DE ENSINO]"
        "<redija aqui seu plano de ensino>"
    ),
    agent=dean
)

# 2. Crew de Setup
crew_setup = Crew(agents=[task_setup_plano.agent], tasks=[task_setup_plano])
input = {
    "topic": topico_geral,
    "specific_topic": topico_especifico
}
plano_de_ensino = crew_setup.kickoff(
    inputs=input
)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Diretor Acadêmico e Orquestrador (Dean)                                                                 │
│                                                                                                                 │
│  Task: Criar o plano de ensino para o curso de 'Deep Learning' sobre o tópico específico de 'Regressão          │
│  Linear'. O plano deve conter os tópicos necessários para o aprendizado, seguindo um raciocínio lógico para     │
│  determinar a progressão, estabelecendo sempre do mais básico ao avançado sobre o tópico específico. Para a     │
│  avaliação é importante criar em memória a lista de tópicos a serem cumpridos do plano de ensino, inicialize    │
│  com ☐ (referente a incomplete). Você possui três status: ✅ (completed), ☐ (incomplete), ❌                    │
│  (learning_problem).                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Diretor Acadêmico e Orquestrador (Dean)                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  [PLANO DE ENSINO]                                                                                              │
│  - ☐ 1. Revisão de Álgebra Linear                                                                               │
│    - Vetores, matrizes, operações básicas, decomposição SVD, produto interno.                                   │
│  - ☐ 2. Conceitos de Estatística Básica                                                                         │
│    - Média, variância, covariância, correlação, distribuição normal.                                            │
│  - ☐ 3. Fundamentos de Regressão Linear Simples                                                                 │
│    - Modelo matemático, hipótese, função de custo (MSE), solução analítica (normal equation).                   │
│  - ☐ 4. Métricas de Avaliação de Modelos de Regressão                                                           │
│    - Erro Quadrático Médio (MSE), Raiz do Erro Quadrático Médio (RMSE), Coeficiente de Determinação (R²).       │
│  - ☐ 5. Regressão Linear Múltipla                                                                               │
│    - Extensão para múltiplas variáveis independentes, interpretação de coeficientes, matriz de                  │
│  características.                                                                                               │
│  - ☐ 6. Regularização de Modelos Lineares                                                                       │
│    - Ridge (L2), Lasso (L1), Elastic Net, efeito da regularização na solução e no viés/variância.               │
│  - ☐ 7. Algoritmo de Gradiente Descendente                                                                      │
│    - Conceitos de batch, stochastic e mini‑batch, taxa de aprendizado, convergência, otimizações (momentum,     │
│  Adam).                                                                                                         │
│  - ☐ 8. Implementação Prática em Python                                                                         │
│    - Uso de NumPy e Pandas para manipulação de dados, implementação do algoritmo do zero, uso do Scikit‑learn   │
│  (LinearRegression, Ridge, Lasso).                                                                              │
│  - ☐ 9. Integração da Regressão Linear em Redes Neurais Profundas                                               │
│    - Camada densa (fully connected) como generalização da regressão linear, inicialização de pesos, papel na    │
│  aprendizagem profunda.                                                                                         │
│  - ☐ 10. Extensões da Regressão Linear                                                                          │
│    - Regressão Polinomial, Feature Engineering (polynomial features, interaction terms), kernels lineares.      │
│  - ☐ 11. Diagnóstico e Problemas Comuns                                                                         │
│    - Underfitting vs. overfitting, análise de resíduos, multicolinearidade, heterocedasticidade, técnicas de    │
│  validação cruzada.                                                                                             │
│  - ☐ 12. Projeto Prático Final                                                                                  │
│    - Aplicação completa: coleta de dataset real, pré‑pr

In [ ]:
plano_de_ensino.raw

'[PLANO DE ENSINO]\n- ☐ 1. Revisão de Álgebra Linear  \n  - Vetores, matrizes, operações básicas, decomposição SVD, produto interno.  \n- ☐ 2. Conceitos de Estatística Básica  \n  - Média, variância, covariância, correlação, distribuição normal.  \n- ☐ 3. Fundamentos de Regressão Linear Simples  \n  - Modelo matemático, hipótese, função de custo (MSE), solução analítica (normal equation).  \n- ☐ 4. Métricas de Avaliação de Modelos de Regressão  \n  - Erro Quadrático Médio (MSE), Raiz do Erro Quadrático Médio (RMSE), Coeficiente de Determinação (R²).  \n- ☐ 5. Regressão Linear Múltipla  \n  - Extensão para múltiplas variáveis independentes, interpretação de coeficientes, matriz de características.  \n- ☐ 6. Regularização de Modelos Lineares  \n  - Ridge (L2), Lasso (L1), Elastic Net, efeito da regularização na solução e no viés/variância.  \n- ☐ 7. Algoritmo de Gradiente Descendente  \n  - Conceitos de batch, stochastic e mini‑batch, taxa de aprendizado, convergência, otimizações (momen

In [ ]:
print("🧠 Memória atual:")
for item in dean_memory.storage.load():
    print(f"- {item['value']} (meta: {item['metadata']})")

🧠 Memória atual:


In [ ]:
# Agente parser
parser_agent = Agent(
    role="Extrator de Dados Focado",
    goal="Extrair um pedaço de informação de um texto, sem adicionar NENHUMA palavra extra.",
    backstory=(
        "Você é um robô de parsing. Você lê um texto e responde *apenas* "
        "com a informação solicitada. Você NUNCA diz 'Aqui está a informação', "
        "'A informação é', ou usa aspas extras."
    ),
    llm=llm,
    verbose=False
)

In [ ]:
# Histórico da conversa para o Dean analisar
conversation_history = [] #[f"Plano de Ensino: {plano_de_ensino}"]
max_turns = 10  # Para evitar loops infinitos
turn = 0

# Mapeia nomes de agentes para os objetos Agente
agents_map = {
    "Professor": professor,
    "Aluno": aluno,
    "Dean": dean
}

# A instrução inicial
next_instruction = (
    "Professor, comece a ensinar o primeiro tópico do plano de ensino ao Aluno, dê prioridade para explorar o "
    "conteúdo do plano de ensino (de forma dissertativa e explicativa) e formular questões de múltipla escolha (até 5 escolhas) "
    "ou questão dissertativa para iniciarmos "
    "a avaliação do aluno e entendermos a profundidade de conhecimento do aluno. Você tem acesso ao plano de ensino abaixo: "
    f"{plano_de_ensino.raw} \n\n"
    "Orientações gerais para explorar na primeira aula: "
    "1. Visão geral do curso; "
    "2. Onde o {specific_topic} se encaixa dentro do {topic}?; "
    "3. Aplicações típicas do {specific_topic} no contexto do {topic}; "
    "4. Por que estudar o {specific_topic}?; "
    "5. Definição de {topic}; "
    "6. Propriedades de {topic}; "
    "7. Casos práticos de implementação em Python; "
    "8. Conclusão; "
    "Exercícios."
)
next_agent_name = "Professor"

while turn < max_turns:
    turn += 1
    print(f"\n--- 🔄 Turno {turn} ---")

    # ---------------------------------
    # SELEÇÃO DO AGENTE E TAREFA
    # ---------------------------------
    if next_agent_name not in agents_map:
        print(f"🚨 Erro: Dean especificou um agente inválido: {next_agent_name}")
        break

    agent_to_run = agents_map[next_agent_name]

    # Cria a tarefa dinâmica
    dynamic_task = Task(
        description=next_instruction,
        expected_output="Sua resposta direta, seguindo seu papel.",
        agent=agent_to_run
    )

    # ---------------------------------
    # EXECUÇÃO DA TAREFA
    # ---------------------------------

    # Cria uma "mini-crew" de 1 agente e 1 tarefa para executar
    task_crew = Crew(agents=[agent_to_run], tasks=[dynamic_task], process=Process.sequential, verbose=0)
    input={
        "topic": topico_geral,
        "specific_topic": topico_especifico
    }
    task_output = task_crew.kickoff(
        inputs=input
    )

    print(f"Resultado:\n{task_output}")
    conversation_history.append(f"{next_agent_name}: {task_output}")

    # ---------------------------------
    # DECISÃO DO DEAN (O Orquestrador)
    # ---------------------------------
    print("\n\n[Dean está pensando...]\n")

    # Cria a tarefa de decisão para o Dean
    # TODO: quebrar em dois agentes (Dean e Orquestrador)
    # Dean:
    # Orquestrador:

    dean_task = Task(
        description=f"""
        Analise o histórico da conversa e o plano de aprendizado.
        O tópico atual é: '{topico_especifico}' do tema de {topico_geral}.
        O plano geral é: {plano_de_ensino}
        Histórico da Conversa:
        ---
        { "\n".join(conversation_history) }
        ---
        Decida a PRÓXIMA AÇÃO. Quem deve falar (Professor ou Aluno) e qual a instrução?
        Ou, se o tópico '{topico_especifico}' foi concluído, defina 'next_agent' como 'Dean'
        e a instrução como 'TÓPICO CONCLUÍDO. Encerrar sessão.'.

        Responda APENAS com um objeto JSON válido, com as chaves "next_agent" e "instruction".
        Exemplo: {{ "next_agent": "Professor", "instruction": "O aluno está confuso sobre 'filtros'. Faça uma analogia." }}
        """,
        expected_output="Um objeto JSON {'next_agent': <next_agent>, 'instruction': <instruction>}.",
        agent=dean
    )

    # Executa a "mini-crew" do Dean
    dean_crew = Crew(agents=[dean], tasks=[dean_task], process=Process.sequential, verbose=0)
    dean_output = dean_crew.kickoff()

    # Parsing com agente 🌟
    # Tarefa 1: Extrair o Agente
    task_get_agent = Task(
        description=f"""
        Analise o texto a seguir:
        ---
        {dean_output}
        ---
        Qual é o valor da chave "next_agent"?
        Responda APENAS com o nome do agente (ex: 'Professor', 'Aluno', ou 'Dean').
        Não inclua aspas ou qualquer outra palavra.
        """,
        expected_output="Um único nome: 'Professor', 'Aluno', ou 'Dean'.",
        agent=parser_agent
    )
    crew_get_agent = Crew(agents=[parser_agent], tasks=[task_get_agent], verbose=0)
    next_agent_name_raw = crew_get_agent.kickoff()

    # Tarefa 2: Extrair a Instrução
    task_get_instruction = Task(
        description=f"""
        Analise o texto a seguir:
        ---
        {dean_output}
        ---
        Qual é o valor da chave "instruction"?
        Responda APENAS com o texto completo da instrução.
        Não inclua "A instrução é:" ou qualquer outra palavra.
        """,
        expected_output="O texto exato da instrução.",
        agent=parser_agent
    )
    crew_get_instruction = Crew(agents=[parser_agent], tasks=[task_get_instruction], verbose=0)
    next_instruction_raw = crew_get_instruction.kickoff()

    # Limpeza final (ainda necessária para garantir)
    next_agent_name = next_agent_name_raw.raw
    next_instruction = next_instruction_raw.raw

    print(f"\n--- [Parser Decidiu] ---")
    print(f"Próximo Agente: {next_agent_name}")
    print(f"Próxima Instrução: {next_instruction}")
    print("------------------------")

    # ---------------------------------
    # VERIFICAÇÃO DE FIM DE LOOP (Agora sem o try/except)
    # ---------------------------------
    if not next_agent_name or not next_instruction or "erro" in next_agent_name.lower():
        print(f"🚨 ERRO: O Agente Parser falhou em extrair dados. Encerrando.")
        print(f"Raw Agent: {next_agent_name_raw}")
        print(f"Raw Instruction: {next_instruction_raw}")
        break

    if next_agent_name.lower() == "dean":
        print(f"--- [Dean] FIM DA SESSÃO: {next_instruction} ---")
        if "concluído" in next_instruction.lower():
            # Assumindo que relatorio_global e topico_especifico estão definidos
            relatorio_global.marcar_aprendido(topico_especifico)
        break


--- 🔄 Turno 1 ---


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Professor Especialista                                                                                  │
│                                                                                                                 │
│  Task: Professor, comece a ensinar o primeiro tópico do plano de ensino ao Aluno, dê prioridade para explorar   │
│  o conteúdo do plano de ensino (de forma dissertativa e explicativa) e formular questões de múltipla escolha    │
│  (até 5 escolhas) ou questão dissertativa para iniciarmos a avaliação do aluno e entendermos a profundidade de  │
│  conhecimento do aluno. Você tem acesso ao plano de ensino abaixo: [PLANO DE ENSINO]                            │
│  - ☐ 1. Revisão de Álgebra Linear                                                                               │
│    - Vetores, matrizes, operações básicas, decomposição SVD, produto interno.                                   │
│  - ☐ 2. Conceitos de Estatística Básica                                                                         │
│    - Média, variância, covariância, correlação, distribuição normal.                                            │
│  - ☐ 3. Fundamentos de Regressão Linear Simples                                                                 │
│    - Modelo matemático, hipótese, função de custo (MSE), solução analítica (normal equation).                   │
│  - ☐ 4. Métricas de Avaliação de Modelos de Regressão                                                           │
│    - Erro Quadrático Médio (MSE), Raiz do Erro Quadrático Médio (RMSE), Coeficiente de Determinação (R²).       │
│  - ☐ 5. Regressão Linear Múltipla                                                                               │
│    - Extensão para múltiplas variáveis independentes, interpretação de coeficientes, matriz de                  │
│  características.                                                                                               │
│  - ☐ 6. Regularização de Modelos Lineares                                                                       │
│    - Ridge (L2), Lasso (L1), Elastic Net, efeito da regularização na solução e no viés/variância.               │
│  - ☐ 7. Algoritmo de Gradiente Descendente                                                                      │
│    - Conceitos de batch, stochastic e mini‑batch, taxa de aprendizado, convergência, otimizações (momentum,     │
│  Adam).                                                                                                         │
│  - ☐ 8. Implementação Prática em Python                                                                         │
│    - Uso de NumPy e Pandas para manipulação de dados, implementação do algoritmo do zero, uso do Scikit‑learn   │
│  (LinearRegression, Ridge, Lasso).                                                                              │
│  - ☐ 9. Integração da Regressão Linear em Redes Neurais Profundas                                               │
│    - Camada densa (fully connected) como generalização da regressão linear, inicialização de pesos, papel na    │
│  aprendizagem profunda.                                                                                         │
│  - ☐ 10. Extensões da Regressão Linear                                                                          │
│    - Regressão Polinomial, Feature Engineering (polynomial features, interaction terms), kernels lineares.      │
│  - ☐ 11. Diagnóstico e Problemas Comuns                                                                         │
│    - Underfitting vs. overfitting, análise de resíduos, multicolinearidade, heterocedasticidade, técnicas de    │
│  validação cruzada.                                    

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Professor Especialista                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **                                                                                                             │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## Aula 1 – Visão Geral e Introdução ao Papel da Regressão Linear no Deep Learning                             │
│                                                                                                                 │
│  ### 1. Visão geral do curso                                                                                    │
│  - **Pergunta para reflexão:** Como vocês imaginam que a sequência de tópicos – de Álgebra Linear até o         │
│  Projeto Prático Final – se interconecta para construir um entendimento sólido de modelos de aprendizado        │
│  profundo?                                                                                                      │
│  - **Pergunta para reflexão:** De que maneira cada módulo (por exemplo, “Algoritmo de Gradiente Descendente”    │
│  ou “Diagnóstico e Problemas Comuns”) pode servir de alicerce para o próximo?                                   │
│                                                                                                                 │
│  ### 2. Onde a Regressão Linear se encaixa dentro do Deep Learning?                                             │
│  - **Pergunta para reflexão:** Se considerarmos uma camada densa (fully‑connected) de uma rede neural, quais    │
│  são as semelhanças matemáticas entre essa camada e um modelo de regressão linear clássico?                     │
│  - **Pergunta para reflexão:** Em que ponto da arquitetura de uma rede profunda a operação linear deixa de ser  │
│  “simples” e passa a ser parte de uma sequência de transformações não‑lineares?                                 │
│                                                                                                                 │
│  ### 3. Aplicações típicas da Regressão Linear no contexto do Deep Learning                                     │
│  - **Pergunta para reflexão:** Quais tipos de problemas (por exemplo, predição de valores contínuos,            │
│  inicialização de pesos, regularização) podem se beneficiar de uma camada linear no início ou no fim de uma     │
│  rede neural?                                                                                                   │
│  - **Pergunta para reflexão:** Como a regressão linear pode ser utilizada como baseline para comparar o         │
│  desempenho de modelos mais complexos?                                                                          │
│                                                                                                                 │
│  ### 4. Por que estudar a Regressão Linear?                                                                     │
│  - **Pergunta para reflexão:** De que forma o entendimento profundo dos parâmetros, da solução analítica        │
│  (equação normal) e das métricas de avaliação (MSE, R²) pode facilitar a interpretação de redes neurais com     │
│  muitas camadas?                                       

Resultado:
**

---

## Aula 1 – Visão Geral e Introdução ao Papel da Regressão Linear no Deep Learning  

### 1. Visão geral do curso  
- **Pergunta para reflexão:** Como vocês imaginam que a sequência de tópicos – de Álgebra Linear até o Projeto Prático Final – se interconecta para construir um entendimento sólido de modelos de aprendizado profundo?  
- **Pergunta para reflexão:** De que maneira cada módulo (por exemplo, “Algoritmo de Gradiente Descendente” ou “Diagnóstico e Problemas Comuns”) pode servir de alicerce para o próximo?  

### 2. Onde a Regressão Linear se encaixa dentro do Deep Learning?  
- **Pergunta para reflexão:** Se considerarmos uma camada densa (fully‑connected) de uma rede neural, quais são as semelhanças matemáticas entre essa camada e um modelo de regressão linear clássico?  
- **Pergunta para reflexão:** Em que ponto da arquitetura de uma rede profunda a operação linear deixa de ser “simples” e passa a ser parte de uma sequência de transformações não‑lineares

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Diretor Acadêmico e Orquestrador (Dean)                                                                 │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Analise o histórico da conversa e o plano de aprendizado.                                              │
│          O tópico atual é: 'Regressão Linear' do tema de Deep Learning.                                         │
│          O plano geral é: [PLANO DE ENSINO]                                                                     │
│  - ☐ 1. Revisão de Álgebra Linear                                                                               │
│    - Vetores, matrizes, operações básicas, decomposição SVD, produto interno.                                   │
│  - ☐ 2. Conceitos de Estatística Básica                                                                         │
│    - Média, variância, covariância, correlação, distribuição normal.                                            │
│  - ☐ 3. Fundamentos de Regressão Linear Simples                                                                 │
│    - Modelo matemático, hipótese, função de custo (MSE), solução analítica (normal equation).                   │
│  - ☐ 4. Métricas de Avaliação de Modelos de Regressão                                                           │
│    - Erro Quadrático Médio (MSE), Raiz do Erro Quadrático Médio (RMSE), Coeficiente de Determinação (R²).       │
│  - ☐ 5. Regressão Linear Múltipla                                                                               │
│    - Extensão para múltiplas variáveis independentes, interpretação de coeficientes, matriz de                  │
│  características.                                                                                               │
│  - ☐ 6. Regularização de Modelos Lineares                                                                       │
│    - Ridge (L2), Lasso (L1), Elastic Net, efeito da regularização na solução e no viés/variância.               │
│  - ☐ 7. Algoritmo de Gradiente Descendente                                                                      │
│    - Conceitos de batch, stochastic e mini‑batch, taxa de aprendizado, convergência, otimizações (momentum,     │
│  Adam).                                                                                                         │
│  - ☐ 8. Implementação Prática em Python                                                                         │
│    - Uso de NumPy e Pandas para manipulação de dados, implementação do algoritmo do zero, uso do Scikit‑learn   │
│  (LinearRegression, Ridge, Lasso).                                                                              │
│  - ☐ 9. Integração da Regressão Linear em Redes Neurais Profundas                                               │
│    - Camada densa (fully connected) como generalização da regressão linear, inicialização de pesos, papel na    │
│  aprendizagem profunda.                                                                                         │
│  - ☐ 10. Extensões da Regressão Linear                                                                          │
│    - Regressão Polinomial, Feature Engineering (polynomial features, interaction terms), kernels lineares.      │
│  - ☐ 11. Diagnóstico e Problemas Comuns                                                                         │
│    - Underfitting vs. overfitting, análise de resíduos, multicolinearidade, heterocedasticidade, técnicas de    │
│  validação cruzada.                                    

...


--- [Parser Decidiu] ---
Próximo Agente: Professor
Próxima Instrução: Apresente as principais métricas de avaliação de modelos de regressão linear: erro quadrático médio (MSE), raiz do erro quadrático médio (RMSE) e coeficiente de determinação (R²). Explique como cada uma é calculada e em que situações são mais adequadas. Em seguida, proponha um exercício: dado um pequeno conjunto de valores reais e previsões, peça ao aluno que calcule o MSE e interprete o resultado.
------------------------

--- 🔄 Turno 5 ---


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Professor Especialista                                                                                  │
│                                                                                                                 │
│  Task: Apresente as principais métricas de avaliação de modelos de regressão linear: erro quadrático médio      │
│  (MSE), raiz do erro quadrático médio (RMSE) e coeficiente de determinação (R²). Explique como cada uma é       │
│  calculada e em que situações são mais adequadas. Em seguida, proponha um exercício: dado um pequeno conjunto   │
│  de valores reais e previsões, peça ao aluno que calcule o MSE e interprete o resultado.                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Professor Especialista                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Vamos conduzir o aprendizado sobre as métricas de avaliação de modelos de regressão linear por meio de         │
│  perguntas que o levem a descobrir cada conceito e sua aplicação.                                               │
│                                                                                                                 │
│  1. **Entendendo o erro quadrático médio (MSE)**                                                                │
│     - Quando comparamos valores reais *y* com valores preditos *ŷ*, como poderíamos quantificar a diferença     │
│  entre eles em cada observação?                                                                                 │
│     - Se quisermos penalizar erros maiores de forma mais forte, que operação matemática (por exemplo, elevar    │
│  ao quadrado) poderíamos aplicar a essas diferenças?                                                            │
│     - Depois de obter esses valores ao quadrado para todas as observações, qual seria a forma mais justa de     │
│  resumir esse conjunto de números? (Pense em soma e divisão.)                                                   │
│     - Como chamaríamos o resultado dessa média dos quadrados dos erros?                                         │
│                                                                                                                 │
│  2. **Explorando a raiz do erro quadrático médio (RMSE)**                                                       │
│     - O MSE tem unidades diferentes das variáveis originais porque envolve o quadrado. Como poderíamos          │
│  transformar o MSE para que a métrica volte a ter as mesmas unidades dos valores *y*?                           │
│     - Qual operação matemática (que “desfaz” o quadrado) você aplicaria ao MSE para obter essa nova métrica?    │
│                                                                                                                 │
│  3. **Investigando o coeficiente de determinação (R²)**                                                         │
│     - Se quisermos saber o quanto da variação total dos valores reais é explicada pelo nosso modelo, que        │
│  comparativo entre duas quantidades de variação poderíamos fazer?                                               │
│     - Qual seria a “variação total” dos dados que devemos considerar (pense em desvio em relação à média)?      │
│     - Como poderíamos medir a variação que ainda resta após a modelagem (pense em resíduos)?                    │
│     - Qual fórmula combina esses dois tipos de variação para gerar um número entre 0 e 1 que indica a           │
│  qualidade do ajuste?                                                                                           │
│                                                                                                                 │
│  4. **Quando usar cada métrica?**                                                                               │
│     - Em que situações seria mais relevante minimizar a magnitude absoluta dos erros (por exemplo, quando       │
│  erros grandes são particularmente indesejáveis)?                                                               │
│     - Quando a interpretabilidade nas mesmas unidades da variável de saída é importante (por exemplo, para      │
│  comunicar resultados a não‑técnicos), qual métrica ser

Resultado:
Vamos conduzir o aprendizado sobre as métricas de avaliação de modelos de regressão linear por meio de perguntas que o levem a descobrir cada conceito e sua aplicação.  

1. **Entendendo o erro quadrático médio (MSE)**  
   - Quando comparamos valores reais *y* com valores preditos *ŷ*, como poderíamos quantificar a diferença entre eles em cada observação?  
   - Se quisermos penalizar erros maiores de forma mais forte, que operação matemática (por exemplo, elevar ao quadrado) poderíamos aplicar a essas diferenças?  
   - Depois de obter esses valores ao quadrado para todas as observações, qual seria a forma mais justa de resumir esse conjunto de números? (Pense em soma e divisão.)  
   - Como chamaríamos o resultado dessa média dos quadrados dos erros?

2. **Explorando a raiz do erro quadrático médio (RMSE)**  
   - O MSE tem unidades diferentes das variáveis originais porque envolve o quadrado. Como poderíamos transformar o MSE para que a métrica volte a ter as mesmas unid

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Diretor Acadêmico e Orquestrador (Dean)                                                                 │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Analise o histórico da conversa e o plano de aprendizado.                                              │
│          O tópico atual é: 'Regressão Linear' do tema de Deep Learning.                                         │
│          O plano geral é: [PLANO DE ENSINO]                                                                     │
│  - ☐ 1. Revisão de Álgebra Linear                                                                               │
│    - Vetores, matrizes, operações básicas, decomposição SVD, produto interno.                                   │
│  - ☐ 2. Conceitos de Estatística Básica                                                                         │
│    - Média, variância, covariância, correlação, distribuição normal.                                            │
│  - ☐ 3. Fundamentos de Regressão Linear Simples                                                                 │
│    - Modelo matemático, hipótese, função de custo (MSE), solução analítica (normal equation).                   │
│  - ☐ 4. Métricas de Avaliação de Modelos de Regressão                                                           │
│    - Erro Quadrático Médio (MSE), Raiz do Erro Quadrático Médio (RMSE), Coeficiente de Determinação (R²).       │
│  - ☐ 5. Regressão Linear Múltipla                                                                               │
│    - Extensão para múltiplas variáveis independentes, interpretação de coeficientes, matriz de                  │
│  características.                                                                                               │
│  - ☐ 6. Regularização de Modelos Lineares                                                                       │
│    - Ridge (L2), Lasso (L1), Elastic Net, efeito da regularização na solução e no viés/variância.               │
│  - ☐ 7. Algoritmo de Gradiente Descendente                                                                      │
│    - Conceitos de batch, stochastic e mini‑batch, taxa de aprendizado, convergência, otimizações (momentum,     │
│  Adam).                                                                                                         │
│  - ☐ 8. Implementação Prática em Python                                                                         │
│    - Uso de NumPy e Pandas para manipulação de dados, implementação do algoritmo do zero, uso do Scikit‑learn   │
│  (LinearRegression, Ridge, Lasso).                                                                              │
│  - ☐ 9. Integração da Regressão Linear em Redes Neurais Profundas                                               │
│    - Camada densa (fully connected) como generalização da regressão linear, inicialização de pesos, papel na    │
│  aprendizagem profunda.                                                                                         │
│  - ☐ 10. Extensões da Regressão Linear                                                                          │
│    - Regressão Polinomial, Feature Engineering (polynomial features, interaction terms), kernels lineares.      │
│  - ☐ 11. Diagnóstico e Problemas Comuns                                                                         │
│    - Underfitting vs. overfitting, análise de resíduos, multicolinearidade, heterocedasticidade, técnicas de    │
│  validação cruzada.                                    